## Uploading Data to and from EC2 to S3 &Basespace using AWS

#### Uploading data to S3 and Creating an IAM role

Select S3 once logged into the AWS console 

Create new bucket

Under Services select IAM, create a new role and choose Amazon EC2 for the role type

Choose AmazonS3FullAccess under policy name

Use the same IAMrole while loading up an instance

Under IAM, create a new User and download the credentials.

Click on the created user;under the permissions tabs;add a new policy- AmazonS3FullAccess.

#### Downloading files into S3 from your local computer using the terminal

pip install awscli

aws configure

aws s3 cp /Users/rashmipoudel/Desktop/upload s3://lab-batch-1 --recursive

#### Downloading files into S3 from basespace directly & AWSCLI configuration

##### SSH into the server

chmod 400 MyFirstKey.pem

ssh -i /Users/cameronstrachan/Desktop/scripts/MyFirstKey.pem -o ServerAliveInterval=60 ubuntu@52.40.146.10

Once logged into the instance:

sudo bash -c "$(curl -L https://basemount.basespace.illumina.com/install)"

mkdir basespace

basemount basespace/

cd basespace 

ls

cd Runs

mkdir data

cp basespace/Runs data/

#### AWS CLI Interface,Configuration and Copying files from EC2 to S3

sudo apt-get update

sudo apt-get install awscli

aws configure

Use the downloaded credential to enter access Key ID and secret access Key.

Access key ID- ** 

Secret Access key- ** 

Region-us-west-2 

Output type-text

aws s3 cp data s3://new-bucket/ --recursive

#### Upload Data from S3 bucket to EC2

sudo mkfs -t ext4 /dev/xvdc

sudo mkdir data

sudo mount /dev/xvdc data

sudo chmod 777 data

cd data

mkdir output

mkdir tools

aws ls s3

To check the storage left on the EC2 instance:
df -h

aws s3 sync s3://metamixis-bucket1 data/input/

aws s3 sync s3://spades-bucket data/input/

mv data/input/12010_x/*.gz data/input/input/

#### Step 2: Downloading and Running  Spades

#### Downloading  Spades and Unzipping files

cd tools

wget http://spades.bioinf.spbau.ru/release3.8.1/SPAdes-3.7.1-Linux.tar.gz

tar -xzf SPAdes-3.8.1-Linux.tar.gz

cd SPAdes-3.8.1-Linux/bin/

cd input

gunzip *.gz




#### Making command strings  

On local machine run the following python script. 

In [ ]:
import os
import errno

try:
    os.makedirs("./input/commands/")
except OSError as exc: 
    if exc.errno == errno.EEXIST and os.path.isdir("./input/commands/"):
        pass
    else:
        raise

filelist = [f for f in os.listdir("./input/") if f.endswith(".gz")]
names = []


for k in range (0,len(filelist)):
    files1=filelist[k]
    files2=files1.split("_")
    names.append(files2[0])

names = sorted(set(names))
names = list(names)


for j in names:
    outfile = open("./input/commands/"+str(j),"w")
    command = []
    for l in range (0,len(filelist)):
        files3=filelist[l]
        files4=files3.split("_")
        if j == files4[0]:
            library = files4[2]
            librarynum = list(library)
            paired = files4[3]
            pairednum = list(paired)
            command.append("--pe" + str(librarynum[3]) + "-" + str(pairednum[1]) + " input/" + filelist[l] + " ")
        else:
            pass
    outfile.write("python tools/SPAdes-3.7.1-Linux/bin/spades.py " + " ".join(command) + "-o output/" + str(j) + "/" + "\n")
    outfile.close()


outfile = open("./input/commands/mkdirectory","w")
for j in names:
    outfile.write("mkdir output/" + str(j) + "\n")
outfile.close()

After running the above script, run the following commands in terminal:  
  
python make_commands2.py  
cd input/commands  
cat mkdirectory *> commands.txt  
sed -i -e 's/.gz//g' commands.txt  
  
In the new terminal:

scp -i /Users/rashmipoudel/Desktop/AWS/MyFirstKey-1.pem ubuntu@50.112.204.164:/home/ubuntu/data/input/input/commands/commands.txt /Users/rashmipoudel/Desktop/

The commands are ready to copy and run on the server. 


#### Running SPAdes on the server
  
1. First, use the mkdir commands to make directories in the output folder. This is where the SPAdes data will be stored.  
2. Open a tmux window by typing tmux 
3. Copy paste the commands, with a space at the bottom, into the tmux window. 
4. Detach from tmux by typing ctrl + b then d
5. Type tmux attach to re-attach to the tmux window.
6. If a second tmux window is opened, they can be toggled between ctrl + b then s. 

#### Step 3: Copying files from EC2 back to the S3 bucket

aws s3 cp data/output/12010-18x  s3://metamixis-bucket1